In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.io import loadmat
import cv2
import skimage as io
import PIL
from PIL import Image
import os
from google.colab.patches import cv2_imshow

In [ ]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

In [ ]:
class BackgroundModel(nn.Module):

	def __init__(self, input_size, hidden_size, num_layers):
		super(BackgroundModel, self).__init__()
		self.rnn = nn.GRU(input_size, hidden_size, num_layers)
		self.linear = nn.Linear(hidden_size, input_size)

	def forward(self, x_in):
		x_in_shape = x_in
		x_input1 = x_in[0].unsqueeze(0)
		latent, _ = self.rnn(x_input1)
		lowrank_es = self.linear(latent).view(x_in_shape.size())
		loss = RMSELoss(lowrank_es, x_input1)
		return lowrank_es, loss


# To check input shape:
video_file = '/content/drive/MyDrive/BMC_videos/111.mp4'
cap = cv2.VideoCapture(video_file)
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)
x.shape

In [ ]:
path_of_org = '/content/drive/MyDrive/Images_org/'
path_of_lo = '/content/drive/MyDrive/Images_lo/'
path_of_sp = '/content/drive/MyDrive/Images_sp/'


model = BackgroundModel(input_size=307200, hidden_size=40, num_layers=2)
optimizer = torch.optim.Rprop(model.parameters(), lr=0.02)
lowrank = torch.FloatTensor(np.zeros([1,307200]))


print('loading data..')

#defining the video file path
video_file = '/content/drive/MyDrive/BMC_videos/111.mp4'
#open the video file
cap = cv2.VideoCapture(video_file)


print('start..')
#read the first frame
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)


# loop over the frames
while ret:


	#pre-processing the frame
	x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)

	#passing frame through model
	lowrank_es, loss = model.forward(x)

  #backpropagation
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()

	ret, frame = cap.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	img_org = x.reshape(480, 640)
	img_org = np.array(img_org, dtype = np.uint8)
	#cv2.imwrite(path_of_org + str(i) + '.jpg', img_org)

	lowrank_result = lowrank_es.squeeze(dim=1)
	img_bg = lowrank_result.reshape(480, 640)
	img_bg = img_bg.detach().numpy()
	img_bg = np.array(img_bg, dtype = np.uint8)
	#cv2.imwrite(path_of_lo + str(i) + '.jpg', img_bg)

	img_fg = cv2.subtract(img_org, img_bg)
	#cv2.imwrite(path_of_sp + str(i) + '.jpg', img_fg)


	img_result = np.concatenate((img_org, img_bg, img_fg), axis=1)
	cv2_imshow(img_result)



#releasing video file
cap.release()



In [ ]:
#creating videos of results:

#dirs_org_frames = os.listdir(path_of_org)
dirs_lo_frames = os.listdir(path_of_lo)
dirs_sp_frames = os.listdir(path_of_sp)

#dirs_org_frames_init = []
dirs_lo_frames_init = []
dirs_sp_frames_init = []

#img_array_org = []
img_array_lo = []
img_array_sp = []


for i in range(x.size(0)):
  #indx_org = dirs_org_frames.index(f'{i}' + '.jpg')
  indx_lo = dirs_lo_frames.index(f'{i}' + '.jpg')
  indx_sp = dirs_sp_frames.index(f'{i}' + '.jpg')

  #dirs_org_frames_init.append(dirs_org_frames[indx_org])
  dirs_lo_frames_init.append(dirs_lo_frames[indx_lo])
  dirs_sp_frames_init.append(dirs_sp_frames[indx_sp])


  #print(path_of_org + dirs_org_frames_init[i])
  print(path_of_lo + dirs_lo_frames_init[i])
  print(path_of_sp + dirs_sp_frames_init[i])


  #img_org = cv2.imread(path_of_org + dirs_org_frames_init[i])
  img_lo = cv2.imread(path_of_lo + dirs_lo_frames_init[i])
  img_sp = cv2.imread(path_of_sp + dirs_sp_frames_init[i])

  #height, width, layers = img_org.shape
  height, width, layers = img_lo.shape
  height, width, layers = img_sp.shape

  size = (width,height)
  #img_array_org.append(img_org)
  img_array_lo.append(img_lo)
  img_array_sp.append(img_sp)


out_org = cv2.VideoWriter('ORA2_org_noRobust.avi',cv2.VideoWriter_fourcc(*'DIVX'), 4, size)
out_lo = cv2.VideoWriter('ICA2_lo_noRobust.avi',cv2.VideoWriter_fourcc(*'DIVX'), 4, size)
out_sp = cv2.VideoWriter('ICA2_sp_noRobust.avi',cv2.VideoWriter_fourcc(*'DIVX'), 4, size)



for i in range(len(img_array_org)):
  out_org.write(img_array_org[i])
out_org.release()


for i in range(len(img_array_lo)):
  out_lo.write(img_array_lo[i])
out_lo.release()


for i in range(len(img_array_sp)):
  out_sp.write(img_array_sp[i])
out_sp.release()